In [1]:
from meerkat.base import time
from meerkat import atlas

In [2]:
# instance device and set output format to .csv (which is default)
dev = atlas.Conductivity(bus_n=1, output='csv')

In [3]:
# single conductivity measurement
dev.measure()

[0.0, 0.0, 0.0, 1.0]

In [4]:
# write 5 samples to .csv file with description
dev.write(description='test_csv', n=5)

In [5]:
# name of file written
dev.writer.path

'2019_04_17_23_12_17_data.csv'

In [6]:
!head $dev.writer.path

#!{"accuracy": null, "bytes": null, "case_sensitive_header": false, "comment": "#", "delimiter": ",", "description": null, "device": {"accuracy": null, "active": null, "bus": "<meerkat.i2c_pi.WrapI2C object at 0x73c01c10>", "calibration_date": null, "description": "", "dtype": null, "error": null, "manufacturer": "Atlas Scientific", "name": "Atlas_Base", "precision": "Varies", "state": null, "urls": "www.atlas-scientific.com", "version_hw": "1.0", "version_sw": "1.0"}, "double_quote": true, "dtypes": null, "encoding": "utf-8", "escape_char": "\\", "format": null, "hash": null, "header": ["description", "sample_n", "conductivity", "total_dissolved_solids", "salinity", "specific_gravity"], "licenses": null, "line_terminator": "\n", "media_type": "text/csv", "name": "Atlas_Conductivity", "null_sequence": "NA", "path": "2019_04_17_23_12_17_data.csv", "precision": null, "quote_char": "\"", "schema": null, "shebang": true, "skip_initial_space": true, "skip_lines": 1, "sources": null, "standa

In [7]:
# Dry and two point calibration
dev.cal_set_dry()
time.sleep(1)
dev.cal_set_low(2000)
time.sleep(1)
dev.cal_set_high(10000)
time.sleep(1)
dev.cal_get()

0

In [8]:
# clear calibration
time.sleep(1)
dev.cal_clear()
time.sleep(1)
dev.cal_get()

0

In [9]:
# probe type
time.sleep(0.5)
dev.set_probe_type(10.0)
time.sleep(0.5)
dev.get_probe_type()

10.0

In [10]:
# temperature compensation
time.sleep(0.5)
dev.temp_set(24.2)
time.sleep(0.5)
dev.temp_get()

24.19

In [11]:
# device information: device type, firmware version
time.sleep(0.5)
dev.info()

('EC', '1.5')

In [12]:
# status of device power: restart code, input voltage Vcc
time.sleep(0.5)
dev.status()

('B', 3.04)

In [13]:
# measurement output formats
dev.output_get()

['EC', 'TDS', 'S', 'SG']

In [14]:
# measure conductivity
time.sleep(0.5)
dev.measure()

[0.0, 0.0, 0.0, 1.0]

In [15]:
# get one sample without a description
dev.get()

['no_description', 0, 0.0, 0.0, 0.0, 1.0]

In [16]:
# get one sample with a description
dev.get('test_1')

['test_1', 0, 0.0, 0.0, 0.0, 1.0]

In [17]:
# get 5 samples with a description
dev.get('test_2', 5)

[['test_2', 0, 0.0, 0.0, 0.0, 1.0],
 ['test_2', 1, 0.0, 0.0, 0.0, 1.0],
 ['test_2', 2, 0.0, 0.0, 0.0, 1.0],
 ['test_2', 3, 0.0, 0.0, 0.0, 1.0],
 ['test_2', 4, 0.0, 0.0, 0.0, 1.0]]

In [18]:
# column names of .csv file and get() method
dev.writer.header

['description',
 'sample_n',
 'conductivity',
 'total_dissolved_solids',
 'salinity',
 'specific_gravity']

In [19]:
# switch to JSON output, csv is default
dev = atlas.Conductivity(bus_n=1, output='json')

In [20]:
# set the metadata publishing interval to every third sample
dev.writer.metadata_interval = 3

In [21]:
# get 7 samples with a description
dev.write(description='test_json', n=7)

In [22]:
# name of file written
dev.writer.path

'2019_04_17_23_12_49_JSON_data.txt'

In [23]:
!head $dev.writer.path

{"conductivity": 0.0, "std_time_ms": "2019-04-17 23:12:49.298750", "sample_n": 0, "total_dissolved_solids": 0.0, "salinity": 0.0, "specific_gravity": 1.0, "description": "test_json"}
{"conductivity": 0.0, "std_time_ms": "2019-04-17 23:12:51.458746", "sample_n": 1, "total_dissolved_solids": 0.0, "salinity": 0.0, "specific_gravity": 1.0, "description": "test_json"}
{"total_dissolved_solids": 0.0, "encoding": "utf-8", "licenses": null, "format": null, "line_terminator": "\n", "schema": null, "hash": null, "escape_char": "\\", "header": ["description", "sample_n", "conductivity", "total_dissolved_solids", "salinity", "specific_gravity"], "comment": "#", "std_time_ms": "2019-04-17 23:12:53.619440", "accuracy": null, "dtypes": null, "quote_char": "\"", "metadata_stream_i": 1, "standard": "RFC 8259", "sources": null, "title": null, "null_sequence": "NA", "specific_gravity": 1.0, "conductivity": 0.0, "media_type": "text/json", "device": {"error": null, "active": null, "manufacturer": "Atlas Sc

In [24]:
# get 5 samples with a description
dev.get('test_2', 7)

['{"conductivity": 0.0, "std_time_ms": "2019-04-17 23:13:04.723495", "sample_n": 0, "total_dissolved_solids": 0.0, "salinity": 0.0, "specific_gravity": 1.0, "description": "test_2"}',
 '{"conductivity": 0.0, "std_time_ms": "2019-04-17 23:13:05.379049", "sample_n": 1, "total_dissolved_solids": 0.0, "salinity": 0.0, "specific_gravity": 1.0, "description": "test_2"}',
 '{"total_dissolved_solids": 0.0, "encoding": "utf-8", "licenses": null, "format": null, "line_terminator": "\\n", "schema": null, "hash": null, "escape_char": "\\\\", "header": ["description", "sample_n", "conductivity", "total_dissolved_solids", "salinity", "specific_gravity"], "comment": "#", "std_time_ms": "2019-04-17 23:13:06.034443", "accuracy": null, "dtypes": null, "quote_char": "\\"", "metadata_stream_i": 0, "standard": "RFC 8259", "sources": null, "title": null, "null_sequence": "NA", "specific_gravity": 1.0, "conductivity": 0.0, "media_type": "text/json", "device": {"error": null, "active": null, "manufacturer": "